<a href="https://colab.research.google.com/github/Dohy-Lee/Do_it-BERT_GPT/blob/main/generation_deploy_colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ratsnlp
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 877 kB/s 
     |████████████████████████████████| 57 kB 6.2 MB/s 
     |████████████████████████████████| 582 kB 67.8 MB/s 
     |████████████████████████████████| 2.8 MB 56.1 MB/s 
     |████████████████████████████████| 141 kB 56.0 MB/s 
     |████████████████████████████████| 419 kB 56.8 MB/s 
     |████████████████████████████████| 596 kB 58.3 MB/s 
     |████████████████████████████████| 880 kB 55.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.1 MB/s 
     |████████████████████████████████| 101 kB 11.7 MB/s 
     |████████████████████████████████| 96 kB 3.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=5f454e9152599bd0b55a67210c02c3c56bad8b35cc49bf49c9c7d3b6d7296666
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Su

##인퍼런스 설정

In [2]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-generation/epoch=1-val_loss=2.29.ckpt


##토크나이저 및 모델 불러오기

In [4]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [3]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
pretrained_model_config = GPT2Config.from_pretrained(
    args.pretrained_model_name,
)
model = GPT2LMHeadModel(pretrained_model_config)
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


##모델 출력값 만들고 후처리

In [5]:
def inference_fn(
        prompt,
        min_length=10,
        max_length=20,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt") #prompt를 토큰화하고 인덱싱한 뒤 파이토치 텐서로 만들기
        with torch.no_grad(): #input_ids 뒤에 이어지는 토큰들 생성
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]]) #generate_ids를 텍스트로 변환
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

In [6]:
!mkdir /root/.ngrok2 && echo "authtoken: 토큰키" > /root/.ngrok2/ngrok.yml

In [ ]:
from ratsnlp.nlpbook.generation import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.generation.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9303-35-203-170-15.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:53:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:53:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:53:25] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:53:30] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:53:49] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:53:57] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:54:03] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:54:13] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:54:19] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:54:24] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:54:29] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2022 04:54:45] "POST /api HTTP/1.1" 200 -
